In [ ]:
import pymatreader as pymr
head_fname = os.path.join(data_dir, 'Hilbert_headmodels', 's011.mat')
head_fname = os.path.join(data_dir, 'headmodel_grid_S01.mat')


fhead =  pymr.read_mat(head_fname)
print("fhead", fhead.keys() )
print("fhead['hdm']", fhead['hdm'].keys() )
print("fhead['hdm']['bnd']", fhead['hdm']['bnd'].keys())
#print("fhead['real_mri']", fhead['real_mri'].keys() )
#print("fhead['real_mri']['hdr']", fhead['real_mri']['hdr'].keys() )
#print("fhead['grid']", fhead['grid'].keys())
#gridpos = fhead['grid']['pos']
#gridpos.shape

fhead['real_mri']['transformorig']

fhead['real_mri']['unit'],fhead['real_mri']['coordsys']

fhead['real_mri']['hdr']['trans']['trans']

fhead['hdm']['bnd']['coordsys'], fhead['hdm']['bnd']['cfg'].keys()

In [ ]:
import bpy
from bpy import context
obj = context.object
mesh = obj.data
mesh

In [ ]:
# get raws

In [ ]:
rawnames = ['S01_off_hold', 'S01_off_move']

from os.path import join as pjoin
raws = []
for rn in rawnames:
    fnf = pjoin(data_dir,rn + '_hires-raw.fif')
    raw = mne.io.read_raw_fif(fnf)
    
    fnf = pjoin(data_dir,rn+'_anns.txt')
    anns = mne.read_annotations(fnf)
    raw.set_annotations(anns)
    raws += [raw]

In [ ]:
tr2 = fhead['real_mri']['transformorig']
tr2 = mne.Transform(tr['from'],tr['to'],tr2)

tr_coreg = mne.read_trans(pjoin('/home/demitau/data_duss/MRI/subjects/','S01_transform.fif'))

#trans = tr_mne
trans = tr_coreg
#trans = tr2
surf = 'head-dense'
surf = 'headsurf.from_ft_surf.pial'
fig = mne.viz.plot_alignment(raw.info, trans=trans, subject='S01',
                             subjects_dir=subjects_dir, surfaces='pial',
                             show_axes=True, dig=True, eeg=[], meg='sensors',
                             coord_frame='meg', mri_fiducials='estimated')
#ValueError: Unknown surface type: ['S01_head']


# if I use headmodels from Hilbert -- need to transform to match freesurfer
# why some coord systems are hardcoded?

In [ ]:
vertices_both_sides = fhead['hdm']['bnd']['pos']
faces_both_sides = fhead['hdm']['bnd']['tri'] - 1

leftInds_coord = np.where(vertices_both_sides[:,0]<= 0)[0]
rightInds_coord = np.where(vertices_both_sides[:,0] > 0)[0]
name2vinds = {'lh':leftInds_coord, 'rh':rightInds_coord}
for name,vinds in name2vinds.items():
    vertices0 = vertices_both_sides[vinds] #* 10
    
    tr = fhead['real_mri']['hdr']['trans']
    tr_mne = mne.Transform(tr['from'],tr['to'],tr['trans'])

    tr_mne_inv = mne.transforms.invert_transform(tr_mne)
    vertices = mne.transforms.apply_trans(tr_mne,vertices0 ) * 10
    #vertices = mne.transforms.apply_trans(tr_mne,vertices0 * 10)
    
    mi = np.min(vinds)
    edges = []
    face_inds = np.where( np.prod( np.isin(faces_both_sides,vinds).astype(int), axis=1) )[0] 
    faces = faces_both_sides[face_inds] - mi
    
    assert np.min(np.diff(vinds)) == 1

    new_mesh = bpy.data.meshes.new('new_mesh')
    new_mesh.from_pydata(vertices, edges, faces)
    new_mesh.update()
    #We created the mesh, but it couldn’t be added to the scene as raw. Only objects could be added to the scene. Let’s make an object with the “new_object” name and link it with the created mesh.
    #Python
    new_object = bpy.data.objects.new('new_object', new_mesh)

    subject = rawname.split('_')[0].upper()

    import bpy
    from bpy import context

    fnf_noext = pjoin(subjects_dir, subject, 'surf', name + '.headsurf.from_ft_surf')
    fnf = fnf_noext + '.obj'

    #obj = context.object
    obj = new_object
    mesh = obj.data

    with open(fnf, 'w') as f:
        f.write("# OBJ file\n")
        for v in mesh.vertices:
            f.write("v %.4f %.4f %.4f\n" % v.co[:])
        for p in mesh.polygons:
            f.write("f")
            for i in p.vertices:
                f.write(" %d" % (i + 1))
            f.write("\n")

    # !obj2srf lh.pial.obj > lh.pial.asc
    # !mris_convert lh.pial.asc lh.pial

    print( fnf_noext )

    from subprocess import run as sprun
    fnf_asc  = fnf_noext + '.asc'
    fnf_pial = fnf_noext + '.pial'

    mriprocscriptsdir = '/home/demitau/current_ownCloud/NeuroSci/MSCA_sci/code/data_proc/mri_preproc'
    mc = pjoin(os.path.expandvars('$FREESURFER_HOME'),'bin', 'mris_convert')
    print('obj2surf')
    r = sprun(['bash', '-c', f'{pjoin(mriprocscriptsdir,"obj2srf") } {fnf} > {fnf_asc}'])
    assert os.path.exists(fnf_asc), fnf_asc
    print('mris_convert')
    r2 = sprun(['bash', '-c', f'{mc} {fnf_asc} {fnf_pial}'])
    assert os.path.exists(fnf_pial), 'file missing  ' + fnf_pial


# !mris_convert lh.pial.asc lh.pial

src = mne.setup_source_space(subject=subject, spacing='all', surface='headsurf.from_ft_surf.pial', 
                       subjects_dir=subjects_dir, add_dist=True, n_jobs=-1, verbose=None)
src.plot(subjects_dir=subjects_dir)

In [ ]:
    print('obj2surf')
    r = sprun(['bash', '-c', f'{pjoin(mriprocscriptsdir,"obj2srf") } {fnf} > {fnf_asc}'])
    print('mris_convert')
    r2 = sprun(['bash', '-c', f'{mc} {fnf_asc} {fnf_pial}'])

In [ ]:
from subprocess import Popen, PIPE
from shlex import split
p1 = Popen(f'{pjoin(mriprocscriptsdir,"obj2srf") } {fnf}'.split(), stdout=PIPE)
#p2 = Popen(split("md5sum"), stdin=p1.stdout)

In [ ]:
f'bash -c {pjoin(mriprocscriptsdir,"obj2srf") } {fnf}  > {fnf_asc}'

In [ ]:
f'bash -c {mc} {fnf_asc} {fnf_pial}'

In [ ]:

#p1 = Popen(f'bash -c {pjoin(mriprocscriptsdir,"obj2srf") } {fnf}'.split(), stdout=PIPE, shell=1)
p1 = Popen(f'bash -c {pjoin(mriprocscriptsdir,"obj2srf") } {fnf}  > {fnf_asc}',  shell=1)
p2 = Popen(f'bash -c {mc} {fnf_asc} {fnf_pial}',
           shell=1)

#s = p1.stdout.read()
#print( s.decode("utf-8") )

#Popen?

In [ ]:
import pymatreader as pymr
#r = pymr.read_mat('cortical_grid.mat')
#template_grid_cm = r['cortical_grid'] * 100

for rawni,rawname_ in enumerate(rawnames):
    sind_str,medcond,task = utils.getParamsFromRawname(rawname_)
    src_ft = h5py.File(src_fname_full, 'r')
    ff = src_ft

    #timeinfo = os.stat(src_fname_full)
    #print('Last mod of src was ', time.ctime(timeinfo.st_mtime) )
    #timeinfo = os.stat(reconst_fname_full)
    #print('Last raw was        ', time.ctime(timeinfo.st_mtime) )

    f = ff[ ff['source_data'][0,0] ]

    #########
    srcCoords_fn = sind_str + '_modcoord_parcel_aal.mat'

    crdf = pymr.read_mat(srcCoords_fn)

    # here we do not have label 'unlabeled'
    labels = crdf['labels']  #indices of sources - labels_corresp_coordance
    coords = crdf['coords_Jan_actual']
    srcgroups_ = crdf['point_ind_corresp']

    #scrgroups_dict = {}
    stcs = []
    #pos_ = f['source_data']['pos'][:,:].T
    
    pos = fhead['grid']['pos']
    numsrc_total = gridpos.shape[0]
    ##########

    defSideFromLabel = False



    ####  Create my
    if sources_type == 'parcel_aal' and defSideFromLabel:
        labels_deford = np.array(labels)[srcgroups_ ]  # because 0 is unlabeled
        ldo = enumerate(labels_deford)
        Lchnis = [labi for labi,lab in ldo if lab.endswith('_L') ]
        Rchnis = [labi for labi,lab in ldo if not lab.endswith('_L') ]
    
        lhi = map(str, Lchnis )
        rhi = map(str, Rchnis )

        concat = Lchnis + Rchnis
        vertices = [Lchnis, Rchnis]
    else:  #define side from coordinate
        leftInds_coord = np.where(pos[:,0]<= 0)[0]
        rightInds_coord = np.where(pos[:,0] > 0)[0]
        vertices = [leftInds_coord, rightInds_coord]

        lhi = map(str, list( vertices[0] ) )
        rhi = map(str, list( vertices[1] ) )

        concat = np.concatenate((leftInds_coord, rightInds_coord))

        #print(labels)

    stc = mne.SourceEstimate(data = srcData, tmin = t0, tstep= tstep  ,
                            subject = sind_str , vertices=vertices)
    stcs += [stc]
